In [1]:
from typing import Optional
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
import os.path as osp
from math import pi as PI
from ase.io import read
import torch
import torch.nn.functional as F
from torch.nn import Embedding, Sequential, Linear, ModuleList
import random
from sklearn.model_selection import train_test_split
from torch_scatter import scatter
from torch_geometric.data.makedirs import makedirs
from torch_geometric.data import download_url, extract_zip, Dataset
from torch_geometric.nn import radius_graph, MessagePassing

In [6]:
import torch
from torch import nn

from torch_geometric.nn.models.schnet import ShiftedSoftplus, SchNet
from torch_geometric.nn import radius_graph, MessagePassing

from torch_scatter import scatter


class EdgeUpdateBlock(nn.Module):
    def __init__(self, in_features, C):
        super(EdgeUpdateBlock, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_features, 2 * C),
            ShiftedSoftplus(),
            nn.Linear(2 * C, C),
        )

    def forward(self, h, edge_attr, edge_index):
        h1 = h[edge_index[0, :]]
        h2 = h[edge_index[1, :]]
        concat = torch.cat((h1, h2, edge_attr), dim=1)
        x = self.mlp(concat)
        return x

    def reset_parameters(self):
        #         torch.nn.init.xavier_uniform_(self.mlp[0].weight)
        self.mlp[0].bias.data.fill_(0)
        #         torch.nn.init.xavier_uniform_(self.mlp[2].weight)
        self.mlp[2].bias.data.fill_(0)


class InteractionBlock(MessagePassing):
    def __init__(self, C):
        super(InteractionBlock, self).__init__()
        self.fc1 = nn.Linear(C, C)
        self.mlp = nn.Sequential(
            nn.Linear(C, C),
            ShiftedSoftplus(),
            nn.Linear(C, C),
            ShiftedSoftplus(),
        )
        self.mlp1 = nn.Sequential(
            nn.Linear(C, C),
            ShiftedSoftplus(),
            nn.Linear(C, C),
        )

    def forward(self, x, edge_attr, edge_index):
        edge = self.mlp(edge_attr)
        h = self.fc1(x)
        edge = self.mlp(edge_attr)
        m = self.propagate(edge_index, x=h, edge_attr=edge)
        m = self.mlp1(m)

        return x + m

    def message(self, x_j, edge_attr):
        return x_j * edge_attr

    def reset_parameters(self):
        #         torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.fc1.bias.data.fill_(0)
        #         torch.nn.init.xavier_uniform_(self.mlp[0].weight)
        self.mlp[0].bias.data.fill_(0)
        #         torch.nn.init.xavier_uniform_(self.mlp[2].weight)
        self.mlp[2].bias.data.fill_(0)
        #         torch.nn.init.xavier_uniform_(self.mlp1[0].weight)
        self.mlp1[0].bias.data.fill_(0)
        #         torch.nn.init.xavier_uniform_(self.mlp1[2].weight)
        self.mlp1[2].bias.data.fill_(0)


class GaussianSmearing(torch.nn.Module):
    def __init__(self, num_gaussians=150, coeff=0.1, m_min=0):
        super(GaussianSmearing, self).__init__()
        offset = torch.arange(0, num_gaussians)
        self.coeff = coeff
        self.inv_coeff = self.coeff ** 0.5
        self.m_min = m_min
        self.register_buffer("offset", offset)

    def forward(self, dist):
        dist = dist.view(-1, 1) - (-self.m_min + self.coeff * self.offset.view(1, -1))
        return torch.exp(-self.inv_coeff * torch.pow(dist, 2))


class EdgeUpdateNet(nn.Module):
    def __init__(
        self, C=32, num_interactions=3, num_gaussians=150, cutoff=10.0, readout="add"
    ):
        super(EdgeUpdateNet, self).__init__()

        self.readout = readout
        self.cutoff = cutoff

        self.embedding = nn.Embedding(100, C)
        self.distance_expansion = GaussianSmearing(num_gaussians)

        self.interactions = nn.ModuleList()
        for _ in range(num_interactions):
            block = InteractionBlock(C)
            self.interactions.append(block)

        self.edge_updates = nn.ModuleList()
        block = EdgeUpdateBlock(num_gaussians + 2 * C, C)
        self.edge_updates.append(block)

        for _ in range(num_interactions - 1):
            block = EdgeUpdateBlock(C * 3, C)
            self.edge_updates.append(block)

        self.mlp = nn.Sequential(
            nn.Linear(C, C // 2),
            ShiftedSoftplus(),
            nn.Linear(C // 2, 1),
        )

        self.reset_parameters()
    def forward(self, data):
        z = data.atomic_numbers.long()
        pos = data.pos
        batch = data.batch


        edge_index = radius_graph(pos, r=self.cutoff, batch=batch)
        row, col = edge_index
        distances = (pos[row] - pos[col]).norm(dim=-1)

  #  def forward(self, z, edge_index, distances, batch=None):
        assert z.dtype == torch.long

        batch = torch.zeros_like(z) if batch is None else batch

        h = self.embedding(z)
        edge_attr = self.distance_expansion(distances)

        for edge_update, interaction in zip(self.edge_updates, self.interactions):
            edge_attr = edge_update(h, edge_attr, edge_index)
            h = interaction(h, edge_attr, edge_index)
        h = self.mlp(h)

        out = scatter(h, batch, dim=0, reduce=self.readout)

        return out

    def reset_parameters(self):
        self.embedding.reset_parameters()
        for interaction in self.interactions:
            interaction.reset_parameters()
        for edge_update in self.edge_updates:
            edge_update.reset_parameters()
        #         torch.nn.init.xavier_uniform_(self.mlp[0].weight)
        self.mlp[0].bias.data.fill_(0)
        #         torch.nn.init.xavier_uniform_(self.mlp[2].weight)
        self.mlp[2].bias.data.fill_(0)


In [8]:
targets=pd.read_csv('targets.csv')
label=targets['energy_per_atom']

In [9]:
structures=pd.read_csv('structures.csv')

,Unnamed: 0,_id
0,0,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
1,1,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
2,2,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
3,3,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
4,4,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
...,...,...
3475,3475,Full Formula (W36 Se70 S1 O1)\nReduced Formula...
3476,3476,Full Formula (W36 Se70 S1 O1)\nReduced Formula...
3477,3477,Full Formula (W36 Se70 S1 O1)\nReduced Formula...
3478,3478,Full Formula (W36 Se70 S1 O1)\nReduced Formula...


In [10]:
atoms_list=np.load('atoms_list.npy', allow_pickle=True)

In [12]:
from torch_geometric.data import Data
import torch
import ase
from pymatgen.io.ase import AseAtomsAdaptor

i=0
data_atoms = []
for atoms in tqdm(atoms_list):
    atoms=AseAtomsAdaptor.get_atoms(atoms)
    # set the atomic numbers, positions, and cell
    atomic_numbers = torch.Tensor(atoms.get_atomic_numbers())
    positions = torch.Tensor(atoms.get_positions())
    natoms = positions.shape[0]
    
    # put the minimum data in torch geometric data object
    data = Data(
        pos=positions,
        atomic_numbers=atomic_numbers,
        natoms=natoms,
    )
    
    # calculate energy
    data.y = label[i]
    i=i+1
    data_atoms.append(data)

100%|██████████| 3480/3480 [00:02<00:00, 1281.40it/s]


In [9]:
from torch_geometric.data import DataLoader
train_dataset, test_dataset = train_test_split(data_atoms, test_size=0.2)
train_loader = DataLoader(train_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)

In [10]:
model=EdgeUpdateNet()
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
#device='cpu'
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 10
#
loss_func = torch.nn.L1Loss() #define loss
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=256)

In [11]:
tr_loss = []
ts_loss=[]#set model to training mode
for epoch in range(200):
    model.train() 
    valid_loss = 0

    train_loss=0

     #shuffle the training data each epoch
    for d in tqdm(train_loader): #go over each training point
        data = d.to(device)#send data to device
       
        out = model(data) 
        optimizer.zero_grad() #zero gradients
        #evaluate data point
        loss = loss_func(out, data.y) #L1 error loss
         #add loss value to aggregate loss
        loss.backward() #compute gradients
        optimizer.step() #apply optimization
        train_loss += loss.item()
    
    model.eval()     # Optional when not using Model Specific layer
    for d in tqdm(test_loader):
        data = d.to(device)
        target = model(data)
        loss = loss_func(target, data.y)
        valid_loss += loss.item()
        
    print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, train_loss/len(train_loader)))
    tr_loss.append(train_loss/len(train_loader))
    print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, valid_loss/len(test_loader)))
    ts_loss.append(valid_loss/len(test_loader))
    
    

  0%|          | 0/2784 [00:00<?, ?it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

100%|██████████| 696/696 [00:05<00:00, 116.98it/s]


Epoch: 000, Average loss: 21.87341
Epoch: 000, Average loss: 3.13777


100%|██████████| 696/696 [00:11<00:00, 60.48it/s]


Epoch: 001, Average loss: 5.98359
Epoch: 001, Average loss: 4.74019


100%|██████████| 696/696 [00:12<00:00, 56.04it/s]


...

100%|██████████| 696/696 [00:03<00:00, 186.05it/s]


Epoch: 186, Average loss: 1.31045
Epoch: 186, Average loss: 1.10648


100%|██████████| 696/696 [00:03<00:00, 186.70it/s]


Epoch: 187, Average loss: 1.16379
Epoch: 187, Average loss: 1.52069


 33%|███▎      | 228/696 [00:01<00:02, 186.17it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Cross Val

In [14]:
from torch_geometric.data import DataLoader
maes=[]
for a in range(7):
    trains=np.where([targets['fold']!=a])[1]
    tests=np.where([targets['fold']==a])[1]
    train_dataset=[data_atoms[i] for i in trains]
    test_dataset=[data_atoms[i] for i in tests]
    train_loader = DataLoader(train_dataset, batch_size=4)
    test_loader = DataLoader(test_dataset, batch_size=4)
    model=EdgeUpdateNet()
    device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
    #device='cpu'
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    epochs = 10
    #
    loss_func = torch.nn.L1Loss() #define loss
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=256)
    tr_loss = []
    ts_loss=[]#set model to training mode
    for epoch in range(100):
        model.train() 
        valid_loss = 0

        train_loss=0

         #shuffle the training data each epoch
        for d in tqdm(train_loader): #go over each training point
            data = d.to(device)#send data to device

            out = model(data) 
            optimizer.zero_grad() #zero gradients
            #evaluate data point
            loss = loss_func(out, data.y) #L1 error loss
             #add loss value to aggregate loss
            loss.backward() #compute gradients
            optimizer.step() #apply optimization
            train_loss += loss.item()

        model.eval()     # Optional when not using Model Specific layer
        for d in tqdm(test_loader):
            data = d.to(device)
            target = model(data)
            loss = loss_func(target, data.y)
            valid_loss += loss.item()

        print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, train_loss/len(train_loader)))
        tr_loss.append(train_loss/len(train_loader))
        
        print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, valid_loss/len(test_loader)))
        ts_loss.append(valid_loss/len(test_loader))
    maes.append(valid_loss/len(test_loader))

    

  0%|          | 0/762 [00:00<?, ?it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

100%|█████████▉| 759/762 [00:37<00:00, 20.65it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

 97%|█████████▋| 106/109 [00:02<00:00, 59.50it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please en

Epoch: 000, Average loss: 1.15836
Epoch: 000, Average loss: 1.46229


100%|██████████| 109/109 [00:02<00:00, 48.74it/s]


Epoch: 001, Average loss: 1.04957
Epoch: 001, Average loss: 1.78722


100%|██████████| 109/109 [00:02<00:00, 50.44it/s]


...

Epoch: 097, Average loss: 0.12102
Epoch: 097, Average loss: 0.27739


100%|██████████| 109/109 [00:01<00:00, 81.03it/s]


Epoch: 098, Average loss: 0.10908
Epoch: 098, Average loss: 0.27499


100%|██████████| 109/109 [00:01<00:00, 70.84it/s] 

Epoch: 099, Average loss: 0.11838
Epoch: 099, Average loss: 0.22893


In [21]:
maes

[0.24836455069379795,
 0.3386210551092384,
 0.1307335179470001,
 0.17127832487533126,
 0.28065828104084783,
 0.24146829436131573,
 0.22892647292123203]

In [17]:
np.mean(maes)

0.23429292813553765

In [20]:
np.std(maes)

0.06325435159957309